In [1]:
# NGSolve
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw
from netgen.webgui import Draw as DrawGeo

# Timing
from time import sleep

# Widgets and Sliders
import ipywidgets as widgets
from ipywidgets import interact, fixed

# Geometry and Mesh

In [2]:
# create geometry
square = MoveTo(0,0).Rectangle(5,1).Face()
hole = Circle((2.5,0.5), 0.2).Face()
hole.edges.name = "circle"
square.edges.Max(Y).name = "top"
square.edges.Min(Y).name = "bottom"
square.edges.Min(X).name = "inlet"
square.edges.Max(X).name = "outlet"
shape = square - hole
#DrawGeo(shape)

In [3]:
# generate mesh out of geometry
geo = OCCGeometry(shape, dim=2)
mesh = Mesh(geo.GenerateMesh(maxh=0.1)).Curve(3)
#Draw(mesh)

# Wall and Circle Deformation

In [4]:
# define finite element space for deformation function
fes_def = VectorH1(mesh,order=3,dirichlet=".*")

# trial and test functions, bilinear form
u_def, v_def = fes_def.TnT()
a_def = BilinearForm(InnerProduct(grad(u_def), grad(v_def))*dx).Assemble()
A_def_inv = a_def.mat.Inverse(freedofs=fes_def.FreeDofs())

# define grid functinos that later
# will hold the deformations
gf_def_top = GridFunction(fes_def) # upper boundary
gf_def_bot = GridFunction(fes_def) # lower boundary
gf_def_rad = GridFunction(fes_def) # radius of hole
gf_def = GridFunction(fes_def) # this will hold all deformations

# initial values for deformations
delta_top = Parameter(-0.005)
delta_bot = Parameter(0.005)
delta_rad = Parameter(0.02)

# update grid functions with actual deformations 
gf_def_top.Set((0, 2*x*(5-x)), definedon=mesh.Boundaries("top"))
gf_def_bot.Set((0, 2*x*(5-x)), definedon=mesh.Boundaries("bottom"))
gf_def_rad.Set(5*CF((x-2.5, y-0.5)), definedon=mesh.Boundaries("circle"))
gf_def_top.vec.data -= A_def_inv@a_def.mat*gf_def_top.vec
gf_def_bot.vec.data -= A_def_inv@a_def.mat*gf_def_bot.vec
gf_def_rad.vec.data -= A_def_inv@a_def.mat*gf_def_rad.vec
# combine all deformations into one grid function
gf_def.vec.data = delta_top.Get() * gf_def_top.vec + \
                  delta_bot.Get() * gf_def_bot.vec + \
                  delta_rad.Get() * gf_def_rad.vec

# Mean Value Deformation (Bend of the Pipe)

In [5]:
mesh.SetDeformation(None)
# define finite element space for deformation function
fes_def_2 = VectorH1(mesh,order=3,dirichlet="inlet")
fes_number = NumberSpace(mesh)
X = fes_def_2 * fes_number

# Parameter here
delta_out = Parameter(0.3)

(u_def_2, lam), (v_def_2, mu) = X.TnT()

a_def_2 = BilinearForm(X)
a_def_2 += InnerProduct(Sym(grad(u_def_2)), Sym(grad(v_def_2)))*dx
a_def_2 += (u_def_2[1]*mu + v_def_2[1]*lam)*ds("outlet")
a_def_2.Assemble()

A_def_inv_2 = a_def_2.mat.Inverse(freedofs=X.FreeDofs())

gf_mean_out = GridFunction(X)
l_def_2 = LinearForm(X)
l_def_2 += 1*mu*ds("outlet")
l_def_2.Assemble()

gf_mean_out.vec.data = A_def_inv_2*l_def_2.vec 

# add deformation to "global" deformation function
gf_def.vec.data += delta_out.Get()*gf_mean_out.components[0].vec

In [6]:
#Draw(gf_mean_out.components[0])
Draw(gf_def, deformation=gf_def)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

BaseWebGuiScene

# Finite Element Setup for Laplace Problem

In [7]:
# define finite element space and 
# grid function for solution of
# a selected problem
fes = H1(mesh, order=3, dirichlet=".*")
gf_sol = GridFunction(fes)

In [8]:
# this function sets up the finite element problem
def solve_problem(solution):
    # inherit finite element space
    # from solution grid function
    fes = solution.space
    
    # fem setup
    u, v = fes.TnT()
    a = BilinearForm(grad(u)*grad(v)*dx).Assemble()
    f = LinearForm(v*dx).Assemble()
    
    # update solution vector
    solution.vec[:] = a.mat.Inverse(freedofs=fes.FreeDofs())*f.vec

In [9]:
# this function updates the deformation based 
# on the values passed by the sliders
def change_grid_layout(delta_val_top, delta_val_bot, delta_val_rad, delta_val_out):
    delta_out.Set(delta_val_out)
    delta_rad.Set(delta_val_rad)
    # update upper deformation
    delta_top.Set(delta_val_top)
    # update lower deformation
    delta_bot.Set(delta_val_bot)
    # update outlet y-shift
    #delta_out_2.Set(delta_val_out_2)
    # apply updated parameters to grid function
    gf_def.vec.data = delta_top.Get() * gf_def_top.vec + \
                      delta_bot.Get() * gf_def_bot.vec + \
                      delta_rad.Get() * gf_def_rad.vec + \
                      delta_out.Get() * gf_mean_out.components[0].vec
    
    # apply deformation function to mesh
    mesh.SetDeformation(gf_def)
    solve_problem(gf_sol)
    scene.Redraw()
    return;

In [10]:
# draw the solution on the mesh
scene=Draw(gf_sol, mesh)

# fun with sliders
interact(change_grid_layout, 
         delta_val_top=widgets.FloatSlider(value=0, min=-0.02, max=0.02, step=0.001, description='upper curve:',
                                           disabled=False, continuous_update=True, orientation='horizontal',
                                           readout=True, readout_format='.3f'), 
         delta_val_bot=widgets.FloatSlider(value=0, min=-0.02, max=0.02, step=0.001, description='lower curve:',
                                           disabled=False, continuous_update=True, orientation='horizontal',
                                           readout=True, readout_format='.3f'),
         delta_val_rad=widgets.FloatSlider(value=0, min=-0.1, max=0.1, step=0.002, description='circle radius:',
                                           disabled=False, continuous_update=True, orientation='horizontal',
                                           readout=True, readout_format='.3f'),
         delta_val_out=widgets.FloatSlider(value=0, min=-1, max=1, step=0.02, description='pipe bend:',
                                           disabled=False, continuous_update=True, orientation='horizontal',
                                           readout=True, readout_format='.3f'))

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

interactive(children=(FloatSlider(value=0.0, description='upper curve:', max=0.02, min=-0.02, readout_format='…

<function __main__.change_grid_layout(delta_val_top, delta_val_bot, delta_val_rad, delta_val_out)>

# Finite Element Setup for Stokes Problem

In [11]:
V = VectorH1(mesh,order=3, dirichlet="inlet|top|bottom|circle")
Q = H1(mesh,order=2)
fes = V*Q

uin = CoefficientFunction( (1.5*4*y*(1.0-y), 0) )

gf_sol = GridFunction(fes)
gf_sol.components[0].Set(uin, definedon=mesh.Boundaries("inlet"))

In [12]:
fes = gf_sol.space    
    
u,p = fes.TrialFunction()
v,q = fes.TestFunction()

nu = 0.001  # viscosity
stokes = (nu*InnerProduct(grad(u), grad(v))+ \
    div(u)*q+div(v)*p - 1e-10*p*q)*dx

a = BilinearForm(stokes).Assemble()
inv_stokes = a.mat.Inverse(fes.FreeDofs(),inverse="sparsecholesky")

# nothing here ...
f = LinearForm(fes).Assemble()

def solve_problem(solution):
    
    a.Assemble()
    f.Assemble()
    #inv_stokes = a.mat.Inverse(fes.FreeDofs(),inverse="sparsecholesky")
    inv_stokes.Update()
    res = f.vec.CreateVector()
    res.data = f.vec - a.mat*solution.vec
    solution.vec.data += inv_stokes * res

In [13]:
def change_grid_layout(delta_val_top, delta_val_bot, delta_val_rad, delta_val_out):
    delta_out.Set(delta_val_out)
    delta_rad.Set(delta_val_rad)
    # update upper deformation
    delta_top.Set(delta_val_top)
    # update lower deformation
    delta_bot.Set(delta_val_bot)
    # update outlet y-shift
    #delta_out_2.Set(delta_val_out_2)
    # apply updated parameters to grid function
    gf_def.vec.data = delta_top.Get() * gf_def_top.vec + \
                      delta_bot.Get() * gf_def_bot.vec + \
                      delta_rad.Get() * gf_def_rad.vec + \
                      delta_out.Get() * gf_mean_out.components[0].vec
    
    # apply deformation function to mesh
    mesh.SetDeformation(gf_def)
    solve_problem(gf_sol)
    scene.Redraw()
    return;

In [14]:
# draw the solution on the mesh
scene=Draw(gf_sol.components[0], mesh)

# fun with sliders
interact(change_grid_layout, 
         delta_val_top=widgets.FloatSlider(value=0, min=-0.02, max=0.02, step=0.001, description='upper curve:',
                                           disabled=False, continuous_update=True, orientation='horizontal',
                                           readout=True, readout_format='.3f'), 
         delta_val_bot=widgets.FloatSlider(value=0, min=-0.02, max=0.02, step=0.001, description='lower curve:',
                                           disabled=False, continuous_update=True, orientation='horizontal',
                                           readout=True, readout_format='.3f'),
         delta_val_rad=widgets.FloatSlider(value=0, min=-0.1, max=0.1, step=0.002, description='circle radius:',
                                           disabled=False, continuous_update=True, orientation='horizontal',
                                           readout=True, readout_format='.3f'),
         delta_val_out=widgets.FloatSlider(value=0, min=-1, max=1, step=0.02, description='pipe bend:',
                                           disabled=False, continuous_update=True, orientation='horizontal',
                                           readout=True, readout_format='.3f'))

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

interactive(children=(FloatSlider(value=0.0, description='upper curve:', max=0.02, min=-0.02, readout_format='…

<function __main__.change_grid_layout(delta_val_top, delta_val_bot, delta_val_rad, delta_val_out)>

# Finite Element Setupt for Navier Stokes Problem

In [17]:
V = VectorH1(mesh,order=3, dirichlet="top|bottom|inlet|circle")
Q = H1(mesh,order=2)
X = V*Q

u,p = X.TrialFunction()
v,q = X.TestFunction()

nu = 0.001  # viscosity
stokes = (nu*InnerProduct(grad(u), grad(v))+ \
    div(u)*q+div(v)*p - 1e-10*p*q)*dx

a = BilinearForm(stokes).Assemble()

# nothing here ...
f = LinearForm(X).Assemble()

# gridfunction for the solution
gfu = GridFunction(X)

In [18]:
uin = CoefficientFunction( (1.5*4*y*(1-y), 0) )
gfu.components[0].Set(uin, definedon=mesh.Boundaries("inlet"))
# Draw (Norm(gfu.components[0]), mesh, "velocity", sd=3)
Draw (gfu.components[0], mesh, "vel");

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

In [19]:
inv_stokes = a.mat.Inverse(X.FreeDofs())

res = f.vec.CreateVector()
res.data = f.vec - a.mat*gfu.vec
gfu.vec.data += inv_stokes * res

Draw (gfu.components[0], mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

BaseWebGuiScene

In [20]:
tau = 0.0008 # timestep

mstar = BilinearForm(X)
mstar += u*v*dx+tau*stokes
mstar.Assemble()
inv = mstar.mat.Inverse(X.FreeDofs(), inverse="sparsecholesky")

In [21]:
conv = BilinearForm(X, nonassemble = True)
conv += (Grad(u) * u) * v * dx

In [22]:
t = 0; i = 0
tend = 1.5
gfut = GridFunction(V, multidim=0)
vel = gfu.components[0]
scene = Draw (gfu.components[0], mesh)
scene.Redraw()
sleep(2)
with TaskManager():
    while t < tend:
        conv.Apply (gfu.vec, res)
        res.data += a.mat*gfu.vec
        gfu.vec.data -= tau * inv * res    

        t = t + tau; i = i + 1
        if i%50 == 0: scene.Redraw()
        #if i%50 == 0: gfut.AddMultiDimComponent(vel.vec)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

# The real fun starts here :-)

In [54]:
mesh.SetDeformation(None)
V = VectorH1(mesh,order=3, dirichlet="top|bottom|inlet|circle")
Q = H1(mesh,order=2)
X = V*Q

u,p = X.TrialFunction()
v,q = X.TestFunction()

nu = 0.001  # viscosity
stokes = (nu*InnerProduct(grad(u), grad(v))+ \
    div(u)*q+div(v)*p - 1e-10*p*q)*dx

a = BilinearForm(stokes).Assemble()

# nothing here ...
f = LinearForm(X).Assemble()

# gridfunction for the solution
gfu = GridFunction(X)

uin = CoefficientFunction( (1.5*4*y*(1-y), 0) )
gfu.components[0].Set(uin, definedon=mesh.Boundaries("inlet"))
# Draw (Norm(gfu.components[0]), mesh, "velocity", sd=3)

inv_stokes = a.mat.Inverse(X.FreeDofs())

res = f.vec.CreateVector()
res.data = f.vec - a.mat*gfu.vec
gfu.vec.data += inv_stokes * res

tau = 0.0006 # timestep

mstar = BilinearForm(X)
mstar += u*v*dx+tau*stokes
mstar.Assemble()
inv = mstar.mat.Inverse(X.FreeDofs(), inverse="sparsecholesky")

conv = BilinearForm(X, nonassemble = True)
conv += (Grad(u) * u) * v * dx
t = 0; i = 0
tend = 0.5
gfut = GridFunction(V, multidim=0)
vel = gfu.components[0]
scene = Draw (gfu.components[0], mesh)
scene.Redraw()
sleep(2)
# solving the problem:
def solve_navier_stokes(solution,time_step,iteration):
    t = time_step
    i = iteration
    tend = 0.5
    a.Assemble()
    f.Assemble()
    inv.Update()
    with TaskManager():
        while t < tend:

            conv.Apply (solution.vec, res)
            res.data += a.mat*gfu.vec
            solution.vec.data -= tau * inv * res    

            t = t + tau; i = i + 1
            if i%50 == 0: scene.Redraw()
            #if i%50 == 0: gfut.AddMultiDimComponent(vel.vec)



WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

In [55]:
def change_grid_layout_time(delta_val_top, delta_val_bot, delta_val_rad, delta_val_out,time_step,iteration):
    SetNumThreads(6)
    with TaskManager(pajetrace=10**8):
        delta_out.Set(delta_val_out)
        delta_rad.Set(delta_val_rad)
        # update upper deformation
        delta_top.Set(delta_val_top)
        # update lower deformation
        delta_bot.Set(delta_val_bot)
        # update outlet y-shift
        #delta_out_2.Set(delta_val_out_2)
        # apply updated parameters to grid function
        gf_def.vec.data = delta_top.Get() * gf_def_top.vec + \
                        delta_bot.Get() * gf_def_bot.vec + \
                        delta_rad.Get() * gf_def_rad.vec + \
                        delta_out.Get() * gf_mean_out.components[0].vec
        
        # apply deformation function to mesh
        mesh.SetDeformation(gf_def)
        solve_navier_stokes(gfu,time_step,iteration)
        scene.Redraw()
    return;

In [56]:
# draw the solution on the mesh
scene=Draw(gfu.components[0], mesh)

# fun with sliders
interact(change_grid_layout_time, 
         delta_val_top=widgets.FloatSlider(value=0, min=-0.02, max=0.02, step=0.001, description='upper curve:',
                                           disabled=False, continuous_update=True, orientation='horizontal',
                                           readout=True, readout_format='.3f'), 
         delta_val_bot=widgets.FloatSlider(value=0, min=-0.02, max=0.02, step=0.001, description='lower curve:',
                                           disabled=False, continuous_update=True, orientation='horizontal',
                                           readout=True, readout_format='.3f'),
         delta_val_rad=widgets.FloatSlider(value=0, min=-0.1, max=0.1, step=0.002, description='circle radius:',
                                           disabled=False, continuous_update=True, orientation='horizontal',
                                           readout=True, readout_format='.3f'),
         delta_val_out=widgets.FloatSlider(value=0, min=-1, max=1, step=0.02, description='pipe bend:',
                                           disabled=False, continuous_update=True, orientation='horizontal',
                                           readout=True, readout_format='.3f'),
        time_step=fixed(0),iteration=fixed(0))

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

interactive(children=(FloatSlider(value=0.0, description='upper curve:', max=0.02, min=-0.02, readout_format='…

<function __main__.change_grid_layout_time(delta_val_top, delta_val_bot, delta_val_rad, delta_val_out, time_step, iteration)>